### PBR Basic
- PBR光照模型的三个条件
    - 基于微平面(Microfacet)的表面模型
    - 能量守恒
    - 应用基于物理的BRDF
- 基于物理的材质+基于物理的光照+基于物理适配的摄像机
- $$h=\frac{l+v}{||l+v||}$$
- 微平面的方向越接近半角向量, 高光反射越尖锐和强烈.![](./Images/roughness.png), 图中可以看出,较高的粗糙度表现出的镜面反射轮廓更大一些,较小的粗糙度表现出的镜面反射轮廓则更小更锐利
- 能量守恒: 出射光线的能量不能超过入射光线的能量, 如图所示, 随着粗糙度的上升镜面反射的区域会增加, 但是镜面反射的强度会下降.当一束光线碰撞到一个表面的时候,它就会分离成一个折射部分和一个反射部分,反射部分就是镜面光照,折射部分就是余下的会进入表面并被吸收的部分,也就是漫反射.
- 辐射通量: 表示一个光源输出的能量$\phi$
- 立体角: 投射到单位球体上的一个截面的面积$I=\frac{d\phi}{d\omega}$ ![](./Images/solid_angle.png)
- 辐射率: 单位面积A中光源在单位立体角所接受到的辐射通量$$L=\frac{d^2\phi}{dAd\omega cos\theta}$$ ![](./Images/radiance.png)
- 如果把立体角和区域A看做无限小, 那么就能用辐射率来表示光线穿过空间中一个点的通量.但事实上,需要计算所有投射到该点的光线的总和,而这个和称为辐照度(**irradiance**)
- 反射率方程计算了半球上的所有入射光的辐射率总和,每一个光线的辐射率都被乘以一个因子$f_{r}$, 这个因子就是BRDF,即使用BRDF基于表面材质属性来对入射辐射率进行加权.
- 使用最多的Cook-Torrance BRDF$$f_{r}=k_{d}f_{lambert}+k_{s}f_{cook-torrance}$$, 其中$k_{d}$表示被折射部分所占的比率,$k_{s}$表示被反射部分所占的比率,BRDF左侧表示的是漫反射分部,称为Lambertian漫反射,$$f_{lambert}=\frac{albedo}{\pi}$$, $$f_{cook-torrance}=\frac{DFG}{4(\omega_{o}\cdot n)(\omega_{i}\cdot n)}$$
- $BxDF$
    - 双向反射分布函数 ($Bidirectional Reflectance Distribution Function, BRDF$)
    - 双向散射分布函数 ($Bidirectional Scattering Distribution Function, BSDF$)
    - 双向透射分布函数 ($Bidirectional Transmittance Distribution Function, BTDF$)
    - 双向散射表面反射分布函数 ($Bidirectional Scattering-Surface Reflectance Distribution Function BSSRDF$)
- 渲染方程
    - 渲染方程的物理基础是能量守恒, 在一个特定的位置和方向, 出射光$L_{o}$是自发光$L_{e}$与反射光线之和,反射光线本身是各个方向的入射光$L_{i}$之和乘以表面反射率以及入射角.
    - 某一点$p$的渲染方程, 可以表示为:$$L_{o}=L_{e}+\int_{\Omega}f_{r}\cdot L_{i}\cdot (\omega_{i}\cdot n)\cdot d\omega_{i}$$ 其中:
        - $L_{o}$是$p$点的出射光亮度
        - $L_{e}$是$p$点发出的光亮度
        - $f_{r}$是$p$点入射方向到出射方向光的反射比例, 即$BxDF$
        - $L_{i}$是$p$点入射光亮度
        - $\omega\cdot n$是入射角带来的入射光衰减
        - $\int_{\Omega}\cdots d\omega_{i}$是入射方向半球积分
<img src="./Images/pbr.jpg" align=center>仅$m = h$的表面点的朝向才会将光线$l$反射到视线$v$的方向，其他表面点对$BRDF$没有贡献
    - $Specular BRDF$ $$f(l, v) = \frac{D(h)F(v,h)G(l,v,h)}{4(n\cdot l)(n\cdot v)}$$, 其中:
        - $D(h)$: 法线分布函数($Normal Distribution Function$), 描述微平面法线分布的概率, 即在表面粗糙度影响下,方向与半角向量方向一致的微表面的数量(概率).
        - $F(l,h)$: 菲涅尔方程($Fresnel Function$), 描述不同的表面角下表面所反射的光线所占的比率
        - $G(l,v,h)$: 几何函数($Geometry Function$), 描述微平面自成阴影的属性,即$m=h$的未被遮蔽的表面点的百分比,当一个平面比较粗糙时,平面上的微平面可能挡住其他微平面从而减少反射光线.
        - $4(n\cdot l)(n\cdot v)$: 校正因子($Correction Factor$), 作为微观几何的局部空间和整个宏观表面的局部空间之间变换的微平面量的校正
   - 法线分布函数($Normal Distribution Function$), 常用$GGX(Trowbridge-Reitz)$ $$D_{GGX(m)} = \dfrac{\alpha^2}{\pi((n\cdot m)^2(\alpha^2-1)+1)^2}$$ 
   因为具有更好的高光长尾<img src="./ndf.jpg" align=center>
   - $Specular F$ 对于菲涅尔项, 一般采用Schlick的Fresnel近似, 因为计算成本低, 且精度足够:
   $$F_{Shlick}(v,h)=F_{0}+(1-F_{0})(1-(v\cdot h))^5$$
   $$F_{0}=vec3(0.04), F_{0}=mix(F_{0}, albedo, metallic)$$
   对于大多数电介质表面而言, 用0.04作为基础反射率已经足够
   - $Specular G$ 常用分离遮蔽阴影($Separable Masking and Shadowing Function$), 该形式将几何项分为两个独立的部分: 光线方向和视线方向, 并对两者用相同的分布函数来描述, UE4的方案是Schlick-GGX, 即基于Schlick近似
           - $$\alpha = (\frac{roughness + 1}{2})^2$$
           - $$k=\frac{\alpha}{2}$$
           - $$G(v)=\frac{n\cdot v}{(n\cdot v)(1-k)+k}$$
           - $$G(l)=\frac{l\cdot v}{(l\cdot v)(1-k)+k}$$
           - $$G(l,v,h) = G(l)\cdot G(v)$$
           
 ### IBL (image based lighting)
 #### Diffuse irrdiance